<a href="https://colab.research.google.com/github/ssvadla/Legal_Text_Classification/blob/main/EM_03_Modular_UL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import f1_score
import math
from sklearn.utils import resample
drive.mount('/content/drive')

#Read the labeled data
train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')
train = train1
train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
for i in train_list:
  train = train.append(i)
train.sort_values("Sentence", inplace = True)
new_train = train.drop_duplicates(subset ="Sentence")
train = new_train
train['Target'].unique()
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()

#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text
len(train['Sentence'])
text = clean_text(train['Sentence'])
tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
37711
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
(4416, 7374)


In [ ]:
#Read the test data
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])

In [ ]:
#Read the unlabeled data
unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
del unlabel['Complete']
del unlabel['Unnamed: 0']

unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

def index_reset(unlabel_2):
  unlabel_2.reset_index(inplace=True)
  del unlabel_2['index']
  return unlabel_2


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def feature_set_selection(train_and_unlabel_words_wo_duplicates,train_words,unlabel_words):
  for term in train_and_unlabel_words_wo_duplicates:
    freq_train = train_words.count(term) / len(train_words)
    freq_unlabel = unlabel_words.count(term) / len(unlabel_words)
    if freq_unlabel == 0:
      PF.append(term)
    elif (freq_train // freq_unlabel) > Threshold_feature:
      PF.append(term)
    else:
      NF.append(term)

In [ ]:

#Identifying  a  set  of  reliable  negative  documents from the unlabeled set
def RN_selection(RN, unlabel_1):
  iteration_RN = 0
  RN_to_be_removed = []
  pos_to_be_removed = []
  freq_each_word_list = []
  Q_pos = []
  count = 0
  if_count = 0
  for doc in unlabel_1['text']:
    doc_words = clean_text(doc)
    doc_words_wo_duplicates = list(set(doc_words))
    for_count = 0
    
    for each_doc_word in doc_words_wo_duplicates:
      for_count = for_count + 1
      freq_each_doc_word = doc_words.count(each_doc_word)
      freq_each_word_list.append(freq_each_doc_word)
      if (freq_each_doc_word > 0) and (each_doc_word in PF):
        if_count = if_count + 1
        pos_to_be_removed.append(count)
        RN_to_be_removed.append(doc)
        break
      
    if for_count == len(doc_words_wo_duplicates):
      Q_pos.append(count)
    count = count + 1
    iteration_RN = iteration_RN + 1
  RN.drop(pos_to_be_removed,axis=0,inplace=True)
  return RN,pos_to_be_removed, Q_pos, if_count



In [ ]:

def classifier_select(train, Q, RN):
  f1_score_list = []
  per_CR_list = []
  loop_variable =0
  while(1):
    p_and_RN = pd.concat([train,RN])
    p_and_RN.reset_index(inplace=True,drop=True)
    p_and_RN_vect = tfidf_vect.transform(p_and_RN['text'])
    p_and_RN_vect_df=pd.DataFrame(p_and_RN_vect.toarray())

    Q_vect = tfidf_vect.transform(Q['text'])
    Q_vect_df=pd.DataFrame(Q_vect.toarray())

    lgb_classifier = lgb.LGBMClassifier()
    lgb_classifier.fit(p_and_RN_vect_df, p_and_RN['Target'])
    np.unique(p_and_RN['Target'])

    #checking the classifier if it gives best results
    train_vect = tfidf_vect.transform(train['text'])
    train_vect_df=pd.DataFrame(train_vect.toarray())
    train_pred = lgb_classifier.predict(train_vect_df)
    classified_negative = (train_pred.tolist()).count(-1)
    percentage = (classified_negative / len(train_pred)) * 100
    model_list.append(lgb_classifier)
    if percentage < percent_thresh:
      thresh_model_list.append(lgb_classifier)
      test_pred = lgb_classifier.predict(t_p.toarray())
      test['Target']= LabelEncoder().fit_transform(test['Target'])
      classification_report_test = classification_report(test['Target'],test_pred,digits=4)
      f1_score_test = f1_score(test['Target'],test_pred,average='weighted')
      f1_score_list.append(f1_score_test)
      per_CR_list.append(classification_report_test)
    Q_pred = lgb_classifier.predict(Q_vect_df)
    np.unique(Q_pred)
    count_q = 0
    total_q = 0 
    out_pos_q = []
    pos_q = []
    for i in Q_pred:
      if i == -1:
        pos_q.append(count_q)
        total_q = total_q + 1
      else:
        out_pos_q.append(count_q)
      count_q = count_q + 1
    Q.reset_index(inplace=True,drop=True)
    W = Q.loc[pos_q,:]
    if W.empty :
      print("W is empty, came out of loop")
      break
    else:
      Q_new = Q.loc[out_pos_q,:]
      Q = Q_new.copy(deep =True)
      RN = pd.concat([RN,W])
      RN.reset_index(inplace=True,drop=True)
      loop_variable = loop_variable + 1
    print("completed iteration")
  result_dict['F1_score'].append(f1_score_list)
  result_dict['CR'].append(per_CR_list)
  return lgb_classifier, model_list , thresh_model_list

In [ ]:
model_list = []
thresh_model_list = []
UL_size_list = []
result_dict = {"F1_score":[],"CR":[],"Unlabel_size":[]}
Threshold_feature = 1
percent_thresh = 5

In [ ]:
unlabel_range = [5000]

In [ ]:

for size in unlabel_range:
  unlabel_1 = unlabel.loc[:size]
  unlabel_1 = index_reset(unlabel_1)
  unlabel_1_copy = unlabel_1.copy(deep = True)
  x_un1 = tfidf_vect.transform(unlabel_1['text'])
  PF = []
  NF = []
  unlabel_1['Target']=-1
  train = train.rename(columns={'Sentence':'text'})
  train['Target']= LabelEncoder().fit_transform(train['Target'])
  train_and_unlabel =  pd.concat([train,unlabel_1])
  print(len(train))
  print(len(unlabel_1))
  print(len(train_and_unlabel))
  train = train.rename(columns={'Sentence':'text'})
  #mixed sampling
  train_and_unlabel =  pd.concat([train,unlabel_1])
  print("len of train_and_unlabel",len(train_and_unlabel))
  train_and_unlabel.reset_index(inplace=True)
  del train_and_unlabel['index']
  del train_and_unlabel['Unnamed: 0']
  target_values = np.unique(train_and_unlabel['Target'].values)
  data_list = []
  data_length_list = []
  for i in target_values:
    df_k = train_and_unlabel[train_and_unlabel['Target']==i]
    data_list.append(df_k)
    data_length_list.append(len(df_k))
  maximum_data = max(data_length_list)
  ratio = math.floor(( 4 / 6 )* maximum_data)
  loop_count = 0
  train_upsampled = []
  for i in data_length_list:
    if i < ratio:
      df_upsampled = resample(data_list[loop_count],replace=True,n_samples=ratio,random_state=123)
    else:
      df_upsampled = resample(data_list[loop_count],replace=True,n_samples=i,random_state=123)
    train_upsampled.append(df_upsampled)
    loop_count = loop_count + 1

  df_res = pd.concat(train_upsampled)
  train_and_unlabel = df_res.copy(deep = True)
  print("len of train_and_unlabel",len(train_and_unlabel))
  train['Target']= LabelEncoder().fit_transform(train['Target'])
  train_and_unlabel_words = clean_text(train_and_unlabel['text'])
  train_and_unlabel_words_wo_duplicates = list(set(train_and_unlabel_words))
  train_words = clean_text(train['text'])
  train_words_wo_duplicates = list(set(train_words))
  unlabel_words = clean_text(unlabel_1['text'])
  unlabel_words_wo_duplicates = list(set(unlabel_words))
  feature_set_selection(train_and_unlabel_words_wo_duplicates,train_words,unlabel_words)
  RN = unlabel_1.copy(deep=True) 
  RN,op_pos_to_be_removed, op_Q_pos, op_if_count = RN_selection(RN,unlabel_1) 
  Q =  unlabel_1.loc[op_pos_to_be_removed,:]
  result_dict['Unlabel_size'].append(size) 
  lgb_classifier, op_model_list, op_thresh_model_list = classifier_select(train, Q, RN)
  

Index(['Unnamed: 0', 'Sentence', 'Target'], dtype='object')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


4416
5001
9417
len of train_and_unlabel 9417
maximum_data 5001
ratio 3334
@ 5001
@ 3334
@ 3334
@ 3334
@ 3334
@ 3334
@ 3334
len of train_and_unlabel 25005
5001
0
#######target unique [-1  0  1  2  3  4  5]
#############train pred [-1  0  1  2  3  4  5]
unique predicted [-1  0  1  2  3  4  5]
classified_negative 675
len(train predic 4416
completed iteration
1
#######target unique [-1  0  1  2  3  4  5]
#############train pred [-1  0  1  2  3  4  5]
unique predicted [-1  0  1  2  3  4  5]
classified_negative 882
len(train predic 4416
completed iteration
2
#######target unique [-1  0  1  2  3  4  5]
#############train pred [-1  0  1  2  3  4  5]
unique predicted [-1  0  1  2  3  4  5]
classified_negative 948
len(train predic 4416
completed iteration
3
#######target unique [-1  0  1  2  3  4  5]
#############train pred [-1  0  1  2  3  4  5]
unique predicted [-1  0  1  2  3  4  5]
classified_negative 971
len(train predic 4416
completed iteration
4
#######target unique [-1  0  1  2  3  4  5]

In [ ]:
with open(r'/content/drive/My Drive/Results/EM/Result01.txt', 'a') as writefile:
        writefile.write("\n")
        writefile.write(" EM Results ")
        writefile.write("\n")
        writefile.write(" final dict ")
        writefile.write("\n")
        writefile.write(str(result_dict))
        writefile.write("\n")
        

In [ ]:

for model in op_model_list:
  test_pred = model.predict(t_p.toarray())
  test['Target']= LabelEncoder().fit_transform(test['Target'])
  classification_report_test = classification_report(test['Target'],test_pred,digits=4)
  print(classification_report_test)

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4615    0.1558    0.2330        77
           1     0.7619    0.6154    0.6809        26
           2     0.7467    0.6292    0.6829       267
           3     0.6944    0.3012    0.4202        83
           4     0.9091    0.2941    0.4444        34
           5     0.6190    0.3824    0.4727        34

    accuracy                         0.4683       521
   macro avg     0.5990    0.3397    0.4192       521
weighted avg     0.6992    0.4683    0.5452       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.5000    0.1299    0.2062        77
           1     0.7500    0.5769    0.6522        26
           2     0.7766    0.5730    0.6595       267
           3     0.6897    0.2410    0.3571        83
           4     0.8182    0.2647    0.4000        34
           5     0.6000 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4545    0.1299    0.2020        77
           1     0.7895    0.5769    0.6667        26
           2     0.7672    0.5431    0.6360       267
           3     0.6667    0.2410    0.3540        83
           4     0.8889    0.2353    0.3721        34
           5     0.5000    0.2941    0.3704        34

    accuracy                         0.3992       521
   macro avg     0.5810    0.2886    0.3716       521
weighted avg     0.6966    0.3992    0.4939       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4762    0.1299    0.2041        77
           1     0.7895    0.5769    0.6667        26
           2     0.7654    0.5131    0.6143       267
           3     0.6800    0.2048    0.3148        83
           4     0.9000    0.2647    0.4091        34
           5     0.5455 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4762    0.1299    0.2041        77
           1     0.7500    0.5769    0.6522        26
           2     0.7824    0.4981    0.6087       267
           3     0.7826    0.2169    0.3396        83
           4     0.7500    0.2647    0.3913        34
           5     0.5000    0.2941    0.3704        34

    accuracy                         0.3743       521
   macro avg     0.5773    0.2829    0.3666       521
weighted avg     0.7150    0.3743    0.4785       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4762    0.1299    0.2041        77
           1     0.7895    0.5769    0.6667        26
           2     0.7706    0.4906    0.5995       267
           3     0.6957    0.1928    0.3019        83
           4     0.7273    0.2353    0.3556        34
           5     0.5000 

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4737    0.1169    0.1875        77
           1     0.7895    0.5769    0.6667        26
           2     0.7574    0.4794    0.5872       267
           3     0.7619    0.1928    0.3077        83
           4     0.8000    0.2353    0.3636        34
           5     0.5000    0.2941    0.3704        34

    accuracy                         0.3570       521
   macro avg     0.5832    0.2708    0.3547       521
weighted avg     0.7038    0.3570    0.4588       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.5500    0.1429    0.2268        77
           1     0.7895    0.5769    0.6667        26
           2     0.7784    0.4869    0.5991       267
           3     0.7083    0.2048    0.3178        83
           4     0.9000    0.2647    0.4091        34
           5     0.5217 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4737    0.1169    0.1875        77
           1     0.7895    0.5769    0.6667        26
           2     0.7647    0.4869    0.5950       267
           3     0.6957    0.1928    0.3019        83
           4     0.8889    0.2353    0.3721        34
           5     0.5238    0.3235    0.4000        34

    accuracy                         0.3628       521
   macro avg     0.5909    0.2760    0.3604       521
weighted avg     0.7043    0.3628    0.4644       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4737    0.1169    0.1875        77
           1     0.7895    0.5769    0.6667        26
           2     0.7711    0.4794    0.5912       267
           3     0.7273    0.1928    0.3048        83
           4     0.8000    0.2353    0.3636        34
           5     0.5000 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for model in op_thresh_model_list:
  test_pred = model.predict(t_p.toarray())
  test['Target']= LabelEncoder().fit_transform(test['Target'])
  classification_report_test = classification_report(test['Target'],test_pred,digits=4)
  print(classification_report_test)